<a href="https://colab.research.google.com/github/noahnisbet/DS-Unit-4-Sprint-1-NLP/blob/main/Copy_of_LS_DS_414_Topic_Modeling_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<img align="left" src="https://lever-client-logos.s3.amazonaws.com/864372b1-534c-480e-acd5-9711f850815c-1524247202159.png" width=200>
<br></br>
<br></br>

# Topic Modeling
## *Data Science Unit 4 Sprint 1 Assignment 4*

Analyze a corpus of Amazon reviews from Unit 4 Sprint 1 Module 1's lecture using topic modeling: 

- Fit a Gensim LDA topic model on Amazon Reviews
- Select appropriate number of topics
- Create some dope visualization of the topics
- Write a few bullets on your findings in markdown at the end
- **Note**: You don't *have* to use generators for this assignment

In [ ]:
#Start Here

In [1]:
import pandas as pd
import spacy
!python -m spacy download en_core_web_lg

url = "https://raw.githubusercontent.com/LambdaSchool/DS-Unit-4-Sprint-1-NLP/main/module1-text-data/data/yelp_coffeeshop_review_data.csv"

shops = pd.read_csv(url)
shops.head()

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_lg')


,coffee_shop_name,full_review_text,star_rating
0,The Factory - Cafe With a Soul,11/25/2016 1 check-in Love love loved the atm...,5.0 star rating
1,The Factory - Cafe With a Soul,"12/2/2016 Listed in Date Night: Austin, Ambia...",4.0 star rating
2,The Factory - Cafe With a Soul,11/30/2016 1 check-in Listed in Brunch Spots ...,4.0 star rating
3,The Factory - Cafe With a Soul,11/25/2016 Very cool decor! Good drinks Nice ...,2.0 star rating
4,The Factory - Cafe With a Soul,12/3/2016 1 check-in They are located within ...,4.0 star rating


In [2]:
import spacy
nlp = spacy.load('en_core_web_lg')

In [3]:
!pip install pyLDAvis

In [4]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# these are all gensim imports 
# check out the gensim docs: https://radimrehurek.com/gensim/
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# we use spacy to load in the word2vec model 
import spacy

# all these imports are for the data viz tool that we use to look at the LDA results and manually determine the topics in our dataset 
# check out the docs: https://pyldavis.readthedocs.io/en/latest/readme.html
# the docs also have an EXAMPLE NOTEBOOK that is more detailed about this data viz tool than our lecture notebook: https://nbviewer.jupyter.org/github/bmabey/pyLDAvis/blob/master/notebooks/pyLDAvis_overview.ipynb
import pyLDAvis
import pyLDAvis.gensim 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
import re

def tokenize(text):
    """Parses a string into a list of semantic units (words)

    Args:
        text (str): The string that the function will tokenize.

    Returns:
        list: tokens parsed out by the mechanics of your choice
    """
    
    # remove non-alphaunmeric
    text = re.sub('(?:(?:[0-9]{2}[:\/,]){2}[0-9]{2,4})', '', text)
    text = re.sub('(?:(?:[0-9]{2}[:\/,])[0-9][:\/,][0-9]{2,4})', '', text)
    text = re.sub('(?:(?:[0-9][:\/,])[0-9]{2}[:\/,][0-9]{2,4})', '', text)
    text = re.sub('(?:(?:[0-9][:\/,])[0-9][:\/,][0-9]{2,4})', '', text)

    text = re.sub('[^a-zA-Z 0-9]', '', text)

    # Remove Emails
    text = re.sub('From: \S+@\S+', '', text)
    # Remove new line characters
    text = re.sub('\s+', ' ', text)
    # Remove non-alphanumeric characters
    text = re.sub('[^0-9 a-zA-Z]+', '', text)
    text = re.sub('[0-9]', '', text)
    
    # make all text lower case
    text = text.lower()
    # split into tokens using whitespaces 
    tokens = text.split(" ")
    
    return tokens

def to_number(text):
  text = re.sub('[A-Z]*[a-z]*', '', text)
  text = re.sub(' ', '', text)
  text = float(text)
  return text

In [3]:
shops['tokens'] = shops['full_review_text'].apply(tokenize)

NameError: ignored

In [29]:
STOP_WORDS = nlp.Defaults.stop_words

In [30]:
tokens = []

""" Update those tokens w/o stopwords"""
for item in shops['tokens']:

  doc_tokens = []

  for token in item:
    if ((token not in (STOP_WORDS)) & (token != '')):
      doc_tokens.append(token)

  tokens.append(doc_tokens)
shops['tokens'] = tokens

In [33]:
#### SPRINT CHALLENGE ####
# For the SC, you will need to create these two objects "id2word" and "corpus" because the LDA model needs them as input

## the LDA model wants id2word and corpus as input 

# Create Dictionary
# pass in tokens 

# check out docs for corpora.Dictionary: https://radimrehurek.com/gensim/corpora/dictionary.html
id2word = corpora.Dictionary(shops['tokens'] )

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in shops['tokens']]

In [42]:

%%time
lda_multicore = gensim.models.ldamulticore.LdaMulticore(corpus=corpus,
                                                        id2word=id2word,
                                                        num_topics = 5, # change num_topics to N, where N is the number of topics in your dataset 
                                                        chunksize=100,
                                                        passes=10,
                                                        per_word_topics=True,
                                                        workers=10) # change workers to be N - 1, where N is the total number of cores that your computer has 
lda_multicore.save('lda_multicore.model')

/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

CPU times: user 28.1 s, sys: 1.7 s, total: 29.8 s
Wall time: 1min


/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for

In [43]:
from gensim import models
lda_multicore =  models.LdaModel.load('lda_multicore.model')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)
/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:418: DeprecationWarning: This function is deprecated.  See https://github.com/RaRe-Technologies/smart_open/blob/develop/MIGRATING_FROM_OLDER_VERSIONS.rst for more information
  warnings.warn(message, category=DeprecationWarning)


In [44]:
print('\nPerplexity: ', lda_multicore.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
# to calculate Coherence, you need to import CoherenceModel from gensim 
coherence_model_lda = CoherenceModel(model=lda_multicore, 
                                     texts=shops['tokens'], 
                                     dictionary=id2word, 
                                     coherence='c_v')

coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt * logsumexp(Elogthetad + Elogbeta[:, int(id)]) for id, cnt in doc)
/usr/local/lib/python3.6/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.
  score += np.sum(cnt


Perplexity:  -7.771802417998807

Coherence Score:  0.3473857971314659


In [45]:
pprint(lda_multicore.print_topics())

[(0,
  '0.013*"food" + 0.011*"tea" + 0.010*"breakfast" + 0.009*"good" + '
  '0.009*"coffee" + 0.009*"sandwich" + 0.008*"great" + 0.007*"place" + '
  '0.006*"like" + 0.006*"salad"'),
 (1,
  '0.037*"coffee" + 0.020*"place" + 0.018*"great" + 0.012*"good" + '
  '0.008*"like" + 0.007*"1" + 0.007*"friendly" + 0.007*"austin" + 0.007*"food" '
  '+ 0.007*"shop"'),
 (2,
  '0.009*"tea" + 0.008*"like" + 0.006*"place" + 0.006*"im" + 0.006*"butter" + '
  '0.006*"service" + 0.005*"order" + 0.005*"time" + 0.005*"got" + '
  '0.005*"cake"'),
 (3,
  '0.024*"coffee" + 0.015*"place" + 0.012*"good" + 0.012*"latte" + '
  '0.010*"like" + 0.008*"sweet" + 0.008*"milk" + 0.008*"gelato" + '
  '0.007*"cream" + 0.007*"love"'),
 (4,
  '0.010*"coffee" + 0.010*"store" + 0.007*"beer" + 0.007*"selection" + '
  '0.006*"live" + 0.005*"austin" + 0.005*"dont" + 0.005*"oak" + 0.005*"like" + '
  '0.005*"good"')]


In [46]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_multicore,
                              corpus, 
                              id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.001142  0.058311       1        1  46.360513
3     -0.052739 -0.015466       2        1  20.488187
0     -0.047540  0.106326       3        1  13.496992
2     -0.089117 -0.116287       4        1  12.250059
4      0.188254 -0.032885       5        1   7.404249, topic_info=           Term         Freq        Total Category  logprob  loglift
355        food  1974.000000  1974.000000  Default  30.0000  30.0000
2534  breakfast   884.000000   884.000000  Default  29.0000  29.0000
312         tea  1386.000000  1386.000000  Default  28.0000  28.0000
246        beer   654.000000   654.000000  Default  27.0000  27.0000
3891   sandwich   462.000000   462.000000  Default  26.0000  26.0000
...         ...          ...          ...      ...      ...      ...
0             1    76.230535  2113.434055   Topic5  -5.8799  -0.7192
462      people    68.568632  1384.404104   Topic5  -5.9859  -0.4021
638      little    66.722823  1435.684300   Topic5  -6.0132  -0.4657
92        great    63.179008  3940.708609   Topic5  -6.0677  -1.5300
276         ive    57.321503  1433.139205   Topic5  -6.1650  -0.6158

[423 rows x 6 columns], token_table=      Topic      Freq  Term
term                       
0         1  0.576313     1
0         2  0.246991     1
0         3  0.060092     1
0         4  0.080911     1
0         5  0.035960     1
...     ...       ...   ...
759       1  0.824280  work
759       2  0.123395  work
759       3  0.032083  work
759       4  0.008226  work
759       5  0.012340  work

[926 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3, 5])

## Stretch Goals

* Incorporate Named Entity Recognition in your analysis
* Incorporate some custom pre-processing from our previous lessons (like spacy lemmatization)
* Analyze a dataset of interest to you with topic modeling